In [1]:
#get data
!mkdir data
!wget -P data https://github.com/Cornell-Quant-Fund/CTC-2024-Derivatives/raw/refs/heads/main/data/cleaned_options_data.zip
!unzip data/cleaned_options_data.zip -d data

--2024-10-12 05:05:55--  https://github.com/Cornell-Quant-Fund/CTC-2024-Derivatives/raw/refs/heads/main/data/cleaned_options_data.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Cornell-Quant-Fund/CTC-2024-Derivatives/refs/heads/main/data/cleaned_options_data.zip [following]
--2024-10-12 05:05:55--  https://raw.githubusercontent.com/Cornell-Quant-Fund/CTC-2024-Derivatives/refs/heads/main/data/cleaned_options_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23864739 (23M) [application/zip]
Saving to: ‘data/cleaned_options_data.zip’

cleaned_options_dat 100%[===================>]  22.7

In [5]:
import pandas as pd
from datetime import datetime
import zipfile
import io
import pytz

# Read the data
with zipfile.ZipFile('data/cleaned_options_data.zip', 'r') as zip_ref:
    with zip_ref.open('cleaned_options_data.csv') as file:
        df = pd.read_csv(file)

# Convert ts_recv to datetime and ensure it's tz-aware
df['ts_recv'] = pd.to_datetime(df['ts_recv'], utc=True)

# Extract information from symbol
df['expiry'] = pd.to_datetime(df['symbol'].str.extract('(\d{6})')[0], format='%y%m%d')
# Make expiry tz-aware
df['expiry'] = df['expiry'].dt.tz_localize('UTC')
df['type'] = df['symbol'].str[-9].map({'C': 'Call', 'P': 'Put'})
df['strike'] = df['symbol'].str[-8:].astype(float) / 1000

# Calculate days till expiry
df['days_till_expiry'] = (df['expiry'] - df['ts_recv']).dt.days


# Reorder columns
column_order = [
    'ts_recv', 'instrument_id', 'symbol', 'type', 'expiry', 'days_till_expiry',
    'strike'
]
df = df[column_order]

In [6]:
df.head(5)

,ts_recv,instrument_id,symbol,type,expiry,days_till_expiry,strike
0,2024-01-02 14:30:02.402838204+00:00,587228771,SPX 240119P04700000,Put,2024-01-19 00:00:00+00:00,16,4700.0
1,2024-01-02 14:30:02.402844171+00:00,587211744,SPX 240119P04300000,Put,2024-01-19 00:00:00+00:00,16,4300.0
2,2024-01-02 14:30:02.402848382+00:00,587211728,SPX 240119P04500000,Put,2024-01-19 00:00:00+00:00,16,4500.0
3,2024-01-02 14:30:02.410507796+00:00,587246038,SPX 240216P04850000,Put,2024-02-16 00:00:00+00:00,44,4850.0
4,2024-01-02 14:30:02.410507796+00:00,587237433,SPX 240216P04650000,Put,2024-02-16 00:00:00+00:00,44,4650.0


In [7]:
# Save the cleaned data to a new CSV file
df.to_csv('data/options_data.csv', index=False)
print("Cleaned data saved to 'data/options_data.csv'")

Cleaned data saved to 'data/options_data.csv'


## Calculating Greeks